# 第6课：探索性数据分析（EDA）

## 学习目标
- 理解 EDA 的目的和流程
- 掌握数据探索的系统方法
- 学会发现数据中的模式和洞察
- 能够进行完整的 EDA 分析

## 1. EDA 简介

探索性数据分析（Exploratory Data Analysis）是在正式建模前对数据进行初步分析的过程，目的是：
- 理解数据结构和特征
- 发现数据中的模式和异常
- 检验假设和直觉
- 为后续分析提供方向

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 设置显示选项
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
sns.set_theme(style="whitegrid")

# 加载数据集
titanic = sns.load_dataset('titanic')
print("Titanic 数据集已加载")
print(f"数据形状: {titanic.shape}")

## 2. 第一步：数据概览

In [ ]:
# 查看前几行
print("数据预览:")
titanic.head(10)

In [ ]:
# 基本信息
print("\n数据基本信息:")
print(titanic.info())

In [ ]:
# 数据形状和列名
print(f"数据形状: {titanic.shape}")
print(f"\n列名: {titanic.columns.tolist()}")
print(f"\n数据类型:")
print(titanic.dtypes)

In [ ]:
# 数值统计摘要
print("数值列统计摘要:")
titanic.describe()

In [ ]:
# 分类列统计摘要
print("分类列统计摘要:")
titanic.describe(include=['object', 'category'])

## 3. 第二步：缺失值分析

In [ ]:
# 缺失值统计
missing = titanic.isnull().sum()
missing_pct = (titanic.isnull().sum() / len(titanic) * 100).round(2)
missing_df = pd.DataFrame({
    '缺失数量': missing,
    '缺失比例(%)': missing_pct
}).sort_values('缺失比例(%)', ascending=False)

print("缺失值分析:")
print(missing_df[missing_df['缺失数量'] > 0])

In [ ]:
# 可视化缺失值
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 缺失值柱状图
missing_cols = missing_df[missing_df['缺失数量'] > 0]
axes[0].barh(missing_cols.index, missing_cols['缺失比例(%)'])
axes[0].set_xlabel('缺失比例 (%)')
axes[0].set_title('各列缺失值比例')

# 缺失值热力图
sns.heatmap(titanic.isnull(), cbar=True, yticklabels=False, ax=axes[1])
axes[1].set_title('缺失值分布热力图')

plt.tight_layout()
plt.show()

## 4. 第三步：单变量分析

In [ ]:
# 目标变量分析：生存率
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# 生存计数
titanic['survived'].value_counts().plot(kind='bar', ax=axes[0], color=['salmon', 'lightgreen'])
axes[0].set_title('生存计数')
axes[0].set_xticklabels(['未生存', '生存'], rotation=0)

# 生存比例
titanic['survived'].value_counts().plot(kind='pie', ax=axes[1], autopct='%1.1f%%',
                                         labels=['未生存', '生存'], colors=['salmon', 'lightgreen'])
axes[1].set_title('生存比例')

plt.tight_layout()
plt.show()

print(f"生存率: {titanic['survived'].mean():.2%}")

In [ ]:
# 数值变量分布
numeric_cols = ['age', 'fare']

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

for i, col in enumerate(numeric_cols):
    # 直方图
    sns.histplot(data=titanic, x=col, kde=True, ax=axes[i, 0])
    axes[i, 0].set_title(f'{col} 分布')
    
    # 箱线图
    sns.boxplot(data=titanic, x=col, ax=axes[i, 1])
    axes[i, 1].set_title(f'{col} 箱线图')

plt.tight_layout()
plt.show()

In [ ]:
# 分类变量分布
cat_cols = ['sex', 'pclass', 'embarked', 'alone']

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

for i, col in enumerate(cat_cols):
    sns.countplot(data=titanic, x=col, ax=axes[i])
    axes[i].set_title(f'{col} 分布')
    
    # 添加百分比标签
    total = len(titanic)
    for p in axes[i].patches:
        percentage = f'{100 * p.get_height() / total:.1f}%'
        axes[i].annotate(percentage, (p.get_x() + p.get_width() / 2., p.get_height()),
                        ha='center', va='bottom')

plt.tight_layout()
plt.show()

## 5. 第四步：双变量分析

In [ ]:
# 生存率 vs 分类变量
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# 性别 vs 生存
sns.barplot(data=titanic, x='sex', y='survived', ax=axes[0, 0])
axes[0, 0].set_title('生存率 by 性别')
axes[0, 0].set_ylabel('生存率')

# 船舱等级 vs 生存
sns.barplot(data=titanic, x='pclass', y='survived', ax=axes[0, 1])
axes[0, 1].set_title('生存率 by 船舱等级')
axes[0, 1].set_ylabel('生存率')

# 登船港口 vs 生存
sns.barplot(data=titanic, x='embarked', y='survived', ax=axes[1, 0])
axes[1, 0].set_title('生存率 by 登船港口')
axes[1, 0].set_ylabel('生存率')

# 是否独自旅行 vs 生存
sns.barplot(data=titanic, x='alone', y='survived', ax=axes[1, 1])
axes[1, 1].set_title('生存率 by 是否独自旅行')
axes[1, 1].set_ylabel('生存率')

plt.tight_layout()
plt.show()

In [ ]:
# 数值变量 vs 生存
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 年龄分布 by 生存
sns.histplot(data=titanic, x='age', hue='survived', kde=True, ax=axes[0])
axes[0].set_title('年龄分布 by 生存')

# 票价分布 by 生存
sns.boxplot(data=titanic, x='survived', y='fare', ax=axes[1])
axes[1].set_title('票价分布 by 生存')

plt.tight_layout()
plt.show()

In [ ]:
# 交叉分析：性别 + 船舱等级 vs 生存
pivot_table = pd.pivot_table(titanic, values='survived', 
                             index='sex', columns='pclass', 
                             aggfunc='mean')

print("生存率交叉表（性别 x 船舱等级）:")
print(pivot_table.round(3))

# 热力图
plt.figure(figsize=(8, 5))
sns.heatmap(pivot_table, annot=True, fmt='.2%', cmap='RdYlGn')
plt.title('生存率热力图（性别 x 船舱等级）')
plt.show()

## 6. 第五步：多变量分析

In [ ]:
# 相关性分析
numeric_titanic = titanic.select_dtypes(include=[np.number])
corr_matrix = numeric_titanic.corr()

plt.figure(figsize=(10, 8))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, cmap='coolwarm', 
            center=0, fmt='.2f', square=True)
plt.title('数值变量相关性矩阵')
plt.show()

In [ ]:
# 成对关系图
sns.pairplot(titanic[['survived', 'age', 'fare', 'pclass']].dropna(), 
             hue='survived', diag_kind='kde', height=2.5)
plt.suptitle('成对关系图', y=1.02)
plt.show()

In [ ]:
# 多维度分析
g = sns.FacetGrid(titanic, col='pclass', row='sex', hue='survived', 
                  height=3, aspect=1.2)
g.map(sns.histplot, 'age', kde=True, alpha=0.6)
g.add_legend()
plt.suptitle('年龄分布（按性别、船舱等级和生存状态）', y=1.02)
plt.show()

## 7. 第六步：特征工程思路

In [ ]:
# 基于 EDA 的特征工程
df = titanic.copy()

# 1. 年龄分组
df['age_group'] = pd.cut(df['age'], bins=[0, 12, 18, 35, 60, 100], 
                         labels=['儿童', '青少年', '青年', '中年', '老年'])

# 2. 家庭规模
df['family_size'] = df['sibsp'] + df['parch'] + 1

# 3. 票价分组
df['fare_group'] = pd.qcut(df['fare'], q=4, labels=['低', '中低', '中高', '高'])

# 4. 头衔提取（从姓名中）
df['title'] = df['name'].str.extract(r' ([A-Za-z]+)\.')

print("新增特征预览:")
print(df[['age', 'age_group', 'sibsp', 'parch', 'family_size', 'fare', 'fare_group', 'title']].head(10))

In [ ]:
# 新特征分析
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# 年龄组 vs 生存
sns.barplot(data=df, x='age_group', y='survived', ax=axes[0, 0])
axes[0, 0].set_title('生存率 by 年龄组')

# 家庭规模 vs 生存
sns.barplot(data=df, x='family_size', y='survived', ax=axes[0, 1])
axes[0, 1].set_title('生存率 by 家庭规模')

# 票价分组 vs 生存
sns.barplot(data=df, x='fare_group', y='survived', ax=axes[1, 0])
axes[1, 0].set_title('生存率 by 票价分组')

# 头衔 vs 生存
top_titles = df['title'].value_counts().head(5).index
sns.barplot(data=df[df['title'].isin(top_titles)], x='title', y='survived', ax=axes[1, 1])
axes[1, 1].set_title('生存率 by 头衔（Top 5）')

plt.tight_layout()
plt.show()

## 8. 第七步：EDA 总结报告

In [ ]:
# 自动生成 EDA 报告
def generate_eda_report(df, target_col=None):
    """生成 EDA 摘要报告"""
    print("=" * 60)
    print("探索性数据分析报告")
    print("=" * 60)
    
    # 1. 数据概览
    print("\n1. 数据概览")
    print("-" * 40)
    print(f"   样本数量: {len(df)}")
    print(f"   特征数量: {len(df.columns)}")
    print(f"   数值特征: {len(df.select_dtypes(include=[np.number]).columns)}")
    print(f"   分类特征: {len(df.select_dtypes(include=['object', 'category']).columns)}")
    
    # 2. 缺失值
    print("\n2. 缺失值情况")
    print("-" * 40)
    missing = df.isnull().sum()
    if missing.sum() > 0:
        for col in missing[missing > 0].index:
            pct = missing[col] / len(df) * 100
            print(f"   {col}: {missing[col]} ({pct:.1f}%)")
    else:
        print("   无缺失值")
    
    # 3. 目标变量
    if target_col and target_col in df.columns:
        print(f"\n3. 目标变量分析 ({target_col})")
        print("-" * 40)
        if df[target_col].dtype in ['int64', 'float64'] and df[target_col].nunique() <= 10:
            for val, count in df[target_col].value_counts().items():
                print(f"   {val}: {count} ({count/len(df)*100:.1f}%)")
    
    # 4. 关键发现
    print("\n4. 数值特征统计")
    print("-" * 40)
    for col in df.select_dtypes(include=[np.number]).columns[:5]:
        print(f"   {col}: 均值={df[col].mean():.2f}, 中位数={df[col].median():.2f}, 标准差={df[col].std():.2f}")
    
    print("\n" + "=" * 60)

# 生成报告
generate_eda_report(titanic, target_col='survived')

## 9. EDA 关键发现总结

基于以上分析，我们发现：

### 关键因素影响生存率
1. **性别**：女性生存率（约74%）远高于男性（约19%）
2. **船舱等级**：一等舱生存率最高，三等舱最低
3. **年龄**：儿童生存率较高
4. **票价**：票价越高，生存率越高

### 特征工程建议
- 创建年龄分组特征
- 创建家庭规模特征
- 提取头衔特征
- 对数变换票价

## 10. 练习题

### 练习：对 tips 数据集进行完整 EDA

In [ ]:
tips = sns.load_dataset('tips')
print(tips.head())

# 在这里进行完整的 EDA 分析
# 1. 数据概览
# 2. 缺失值检查
# 3. 单变量分析
# 4. 双变量分析（小费与各因素的关系）
# 5. 关键发现总结


## 11. 本课小结

EDA 流程：
1. **数据概览**：shape、info、describe
2. **缺失值分析**：统计和可视化
3. **单变量分析**：分布、频率
4. **双变量分析**：相关性、分组比较
5. **多变量分析**：交叉表、多维可视化
6. **特征工程**：基于发现创建新特征
7. **总结报告**：记录关键发现